This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np
import os
os.environ['LOG_LEVEL'] = 'INFO'
import glob
from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
SUFFIX = '20180221'
file_list = glob.glob('../../dmu32/dmu32_SGP/data/tiles/sgp_{}_cigale_*.fits'.format(SUFFIX))

In [3]:
#len(master_catalogue)

# Best sources

Define a good far-IR measurement as:
- an existing flux in the band;
- the flag from XID+ must not be set;
- the signal to noise ratio must be over 2.

In [4]:
#good = {}
#for band in ['pacs_green', 'pacs_red', 'spire_250', 'spire_350', 'spire_500']:
#    good[band] = (~np.isnan(master_catalogue['f_{}'.format(band)]) & 
#                                   ~master_catalogue['flag_{}'.format(band)])
#    good[band][good[band]] &= (master_catalogue[good[band]]['f_{}'.format(band)] /
#                              master_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)

We will keep only sources with at leat 2 good far-IR measurements (we may actually use less sources are not all may have a redshift).

In [5]:
# combined_good = np.sum(list(good.values()), axis=0) >= 2

In [6]:
# print("Number of good sources: {}".format(np.sum(combined_good)))

# Main catalogue for CIGALE

In [7]:
# best_catalogue = master_catalogue[combined_good].copy()

In [8]:
# # Correction for galactic extinction
# best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [9]:
# # Convertion to CIGALE format
# best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

## Band selection

We want to use only one filter for similar bands. We define an order of preference and set to NaN the flux in the lower prefered bands when a prefered band is available. Some band may have a 0 flux, we set there values to NaN.

In [10]:
u_bands = ["omegacam_u"]
g_bands = ["omegacam_g", "decam_g",            "gpc1_g"]
r_bands = ["omegacam_r", "decam_r",            "gpc1_r"]
i_bands = ["omegacam_i", "decam_i",            "gpc1_i"]
z_bands = [              "decam_z", "vista_z", "gpc1_z"]
y_bands = [              "decam_y", "vista_y", "gpc1_y"]

def remove_unneeded_fluxes(list_of_bands):
    for band_idx, band in enumerate(list_of_bands[:-1]):
        mask = ~np.isnan(best_catalogue[band])
        for lower_band in list_of_bands[band_idx+1:]:
            best_catalogue[lower_band][mask] = np.nan
            best_catalogue["{}_err".format(lower_band)][mask] = np.nan

In [11]:
# remove_unneeded_fluxes(g_bands)
# remove_unneeded_fluxes(u_bands)
# remove_unneeded_fluxes(r_bands)
# remove_unneeded_fluxes(i_bands)
# remove_unneeded_fluxes(z_bands)
# remove_unneeded_fluxes(y_bands)

In [12]:
non_band_flags = ['flag_cigale', 
                  'flag_cigale_opt', 
                  'flag_cigale_ir', 
                  'flag_cigale_ironly', 
                  'flag_gaia', 
                  'flag_cleaned', 
                  'flag_merged', 
                  'flag_optnir_obs', 
                  'flag_optnir_det']

In [13]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")
n_zphot = 0
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)
    
    #Temp solution to problem caused by flag unit = mag:
    for col in best_catalogue.colnames:
        if col.startswith('flag_') and (col not in non_band_flags):
            best_catalogue[col].unit = None
            best_catalogue[col] = best_catalogue[col].astype(bool)
            
    
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    good = {}
    for band in [ 'spire_250', 'spire_350', 'spire_500']:
        good[band] = (~np.isnan(best_catalogue['f_{}'.format(band)]) & 
                                       ~best_catalogue['flag_{}'.format(band)])
        good[band][good[band]] &= (best_catalogue[good[band]]['f_{}'.format(band)] /
                                  best_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)
    
    combined_good = np.sum(list(good.values()), axis=0) >= 2
    
    best_catalogue = best_catalogue[combined_good]
    #best_catalogue = best_catalogue[optnir]


    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]
    n_zphot += len(best_catalogue)
    

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)

    #best_catalogue.write("data_tmp/tiles/SGP_cigale_optnir_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    best_catalogue.write("data_tmp/tiles/SGP_cigale_best_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    
print("Number of zphot optnir objects: {}".format(n_zphot))

Number of sources with z-spec in file 265: 1373
Number of sources with z-spec in file 54: 772
Number of sources with z-spec in file 182: 1219
Number of sources with z-spec in file 120: 1607
Number of sources with z-spec in file 111: 1544
Number of sources with z-spec in file 254: 992
Number of sources with z-spec in file 65: 1534
Number of sources with z-spec in file 173: 1332
Number of sources with z-spec in file 294: 1215
Number of sources with z-spec in file 138: 590
Number of sources with z-spec in file 236: 1259


INFO:herschelhelp.external:For 1 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 36: 1275
Number of sources with z-spec in file 207: 1233
Number of sources with z-spec in file 109: 1800
Number of sources with z-spec in file 94: 1612
Number of sources with z-spec in file 142: 590
Number of sources with z-spec in file 9: 1137
Number of sources with z-spec in file 115: 1529
Number of sources with z-spec in file 88: 1624


INFO:herschelhelp.external:For 1 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 61: 819
Number of sources with z-spec in file 250: 991
Number of sources with z-spec in file 186: 1194


...

Number of sources with z-spec in file 277: 718
Number of sources with z-spec in file 132: 425
Number of sources with z-spec in file 179: 1083
Number of sources with z-spec in file 148: 273
Number of sources with z-spec in file 103: 1491
Number of sources with z-spec in file 3: 868
Number of sources with z-spec in file 77: 1688
Number of sources with z-spec in file 246: 1072
Number of sources with z-spec in file 286: 1034
Number of sources with z-spec in file 161: 953
Number of sources with z-spec in file 224: 1312
Number of sources with z-spec in file 188: 1061
Number of sources with z-spec in file 15: 1798
Number of sources with z-spec in file 215: 996
Number of sources with z-spec in file 24: 1535
Number of sources with z-spec in file 150: 0
Number of sources with z-spec in file 86: 1749
Number of sources with z-spec in file 107: 1447
Number of sources with z-spec in file 7: 972
Number of sources with z-spec in file 209: 1172
Number of sources with z-spec in file 38: 1547


INFO:herschelhelp.external:For 1 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 242: 1216
Number of sources with z-spec in file 73: 1048
Number of sources with z-spec in file 273: 1056
Number of sources with z-spec in file 42: 1633
Number of sources with z-spec in file 194: 1195
Number of sources with z-spec in file 238: 1351
Number of sources with z-spec in file 136: 632
Number of sources with z-spec in file 20: 1199
Number of sources with z-spec in file 211: 1018
Number of sources with z-spec in file 82: 1350


INFO:herschelhelp.external:For 1 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 154: 745
Number of sources with z-spec in file 165: 969
Number of sources with z-spec in file 282: 1274
Number of sources with z-spec in file 11: 1727
Number of sources with z-spec in file 220: 1186
Number of zphot optnir objects: 352804


In [14]:
#best_catalogue.write("data_tmp/CDFS-SWIRE_cigale_best_extcor_20180129.fits")
n_zphot = 0
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)
    
    #Temp solution to problem caused by flag unit = mag:
    for col in best_catalogue.colnames:
        if col.startswith('flag_') and (col not in non_band_flags):
            best_catalogue[col].unit = None
            best_catalogue[col] = best_catalogue[col].astype(bool)
            
    



    best_catalogue = best_catalogue[~np.isnan(best_catalogue["redshift"])]
    n_zphot += len(best_catalogue)
    

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)

    #best_catalogue.write("data_tmp/tiles/SGP_cigale_optnir_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    best_catalogue.write("data_tmp/tiles/SGP_cigale_all_extcor_{}_{}.fits".format(SUFFIX, file_num), overwrite=True)
    
print("Number of zphot optnir objects: {}".format(n_zphot))

Number of sources with z-spec in file 265: 49825


INFO:herschelhelp.external:For 13 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 6 sources, the band omegacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 6 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 5 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the

Number of sources with z-spec in file 54: 36011


INFO:herschelhelp.external:For 16 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 3 sources, the band omegacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 2 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.


Number of sources with z-spec in file 182: 51009


...

Number of sources with z-spec in file 11: 67884


INFO:herschelhelp.external:For 31 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band omegacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 10 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 4 sources, 

Number of sources with z-spec in file 220: 49595


INFO:herschelhelp.external:For 19 sources, the band omegacam_u should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band omegacam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band decam_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 8 sources, the band gpc1_g should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 5 sources, the band gpc1_r should not be used because it overlaps or is below the Lyman limit at the redshift of these sources. These fluxes were set to NaN.
INFO:herschelhelp.external:For 5 sources, the

Number of zphot optnir objects: 17054138


# Catalogue using spectroscopic redshift

In [15]:
n_zspec = 0
for tile in file_list:
        
    file_num = tile.split('_')[4].split('.')[0]
    best_catalogue = Table.read(tile)
    
    #Temp solution to problem caused by flag unit = mag:
    for col in best_catalogue.colnames:
        if col.startswith('flag_') and (col not in non_band_flags):
            best_catalogue[col].unit = None
            best_catalogue[col] = best_catalogue[col].astype(bool)
    
    
    
    
    optnir = ((best_catalogue['flag_optnir_det'] == 3) 
          | (best_catalogue['flag_optnir_det'] == 7))
    
    good = {}
    for band in [ 'spire_250', 'spire_350', 'spire_500']:
        good[band] = (~np.isnan(best_catalogue['f_{}'.format(band)]) & 
                                       ~best_catalogue['flag_{}'.format(band)])
        good[band][good[band]] &= (best_catalogue[good[band]]['f_{}'.format(band)] /
                                  best_catalogue[good[band]]['ferr_{}'.format(band)] >= 2)
    
    combined_good = np.sum(list(good.values()), axis=0) >= 2
    
    best_catalogue = best_catalogue[combined_good]
    #best_catalogue = best_catalogue[optnir]


    best_catalogue = best_catalogue[~np.isnan(best_catalogue["zspec"])]
    best_catalogue.remove_column('redshift')
    best_catalogue["zspec"].name = "redshift"
    n_zspec += len(best_catalogue)

    print("Number of sources with z-spec in file {}: {}".format(file_num, len(best_catalogue)))

    # Correction for galactic extinction
    best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

    # Convertion to CIGALE format

    best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True, remove_zerofluxes=True)

    remove_unneeded_fluxes(g_bands)
    remove_unneeded_fluxes(u_bands)
    remove_unneeded_fluxes(r_bands)
    remove_unneeded_fluxes(i_bands)
    remove_unneeded_fluxes(z_bands)
    remove_unneeded_fluxes(y_bands)


    best_catalogue.write("data_tmp/tiles/SGP_cigale_best_extcor_zspec_{}_{}.fits".format(SUFFIX, file_num), 
                         overwrite=True)
    
print("Number of zspec optnir objects: {}".format(n_zspec))

Number of sources with z-spec in file 265: 12
Number of sources with z-spec in file 54: 16
Number of sources with z-spec in file 182: 30
Number of sources with z-spec in file 120: 33
Number of sources with z-spec in file 111: 39
Number of sources with z-spec in file 254: 14
Number of sources with z-spec in file 65: 40
Number of sources with z-spec in file 173: 17
Number of sources with z-spec in file 294: 57
Number of sources with z-spec in file 138: 5
Number of sources with z-spec in file 236: 36
Number of sources with z-spec in file 36: 27
Number of sources with z-spec in file 207: 33
Number of sources with z-spec in file 109: 38
Number of sources with z-spec in file 94: 35
Number of sources with z-spec in file 142: 14
Number of sources with z-spec in file 9: 26
Number of sources with z-spec in file 115: 36
Number of sources with z-spec in file 88: 36
Number of sources with z-spec in file 61: 22
Number of sources with z-spec in file 250: 41
Number of sources with z-spec in file 186: 